If any problems occur, please refer to the E2_Vector_Search to check out the set up process. 

In [3]:
# Import Vertex AI SDK
PROJECT_ID = !gcloud config get project
PROJECT_ID = PROJECT_ID.n
LOCATION = "europe-west2"
LOCATION_DEPLOY = "europe-west2" #Location to deploy GCP resources

import vertexai
from google.cloud import aiplatform

vertexai.init(project=PROJECT_ID, location=LOCATION)

2023-12-05 14:56:57.565120: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-05 14:56:58.755032: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64
2023-12-05 14:56:58.755190: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/loca

In [13]:
from vertexai.preview.language_models import ChatModel

""" 
between using the most stable model in the last 6 month: chat-bison@001 
and the latest, most capable model, there is a massive difference in 
terms of context awareness, length of answer, ability to follow instructions,
etc. How do we measure these using metrics? I'm working on it.

"""
class PaLMWrapper:
    def __init__(self):
        self.chat_model = ChatModel.from_pretrained("chat-bison")
        self.parameters = {
            "temperature": 0.3,
            "max_output_tokens": 256,
            "top_p": 0.95,
        }
    
    def generate_response(self, context, message):
        chat = self.chat_model.start_chat(context=context)
        response = chat.send_message(message, **self.parameters)
        return response.text

In [14]:
pw = PaLMWrapper()
response = pw.generate_response(context='you are a london weather forecaster', message='whats the weather like today')
response

' Today will be cloudy with a high of 55 degrees Fahrenheit and a low of 40 degrees Fahrenheit. There is a 30% chance of rain.'

In [15]:
import pandas as pd

import vertexai
from vertexai.preview.language_models import TextEmbeddingModel
from google.cloud import aiplatform

class VertexAIVectorStore:
    def __init__(self):
        
        # god awful way of doing it, should be a config and passed through but oh well hacky hack
        self.gen_ai_index_endpoint = aiplatform.MatchingEngineIndexEndpoint(
            index_endpoint_name="projects/playpen-8d8611/locations/europe-west2/indexEndpoints/5891412000042385408"
        )

        self.gen_ai_index = aiplatform.MatchingEngineIndex(
            index_name="projects/playpen-8d8611/locations/europe-west2/indexes/2680908415680643072"
        )
        
        self.model = TextEmbeddingModel.from_pretrained("textembedding-gecko@001")
        self.df = pd.read_csv('text_data_g_embedding.csv')
        
    def search(self, input, k=3):
        embedding_vec =  self.model.get_embeddings([input])[0].values #Send request to embedding model to generate the embedding vector

        #find neighbours using vector search
        neighbours = self.gen_ai_index_endpoint.find_neighbors(
            deployed_index_id="gen_ai_deployed_index",
            queries=[embedding_vec],
            num_neighbors=k,
        )[0]
        
        results = []
        for nb in neighbours:
            text_id = nb.id
            text = self.df.iloc[int(nb.id)]['text']
            score = nb.distance
            results.append((text_id, text, score))
        
        return results


In [7]:
vai = VertexAIVectorStore()

[('41',
  'Security & fraudLost or stolen cardsReport a lost or stolen Business CardReport a lost or stolen Business payment cardReport a lost or stolen Corporate CardReport a lost or stolen Corpoarate cardReport a fraudReport a fraud on our business accountsMake a complaintReport fraudulent use of online bankingStaying safeProtect your business from fraudManage the cyber threat to your business',
  0.7093294858932495),
 ('55',
  "Report\xa0fraud on online business bankingCall us immediately\xa0 if you think any of your Lloyds Business Bank accounts have been:accessed by a third party\xa0ortargeted for\xa0fraud.Please get in touch even if you haven't suffered any loss from the attempted fraud.This page is for business customersFind out how to report fraud if you’re a personal customerReport FraudBy PhoneOnline for Business\xa00800 056 3099Commercial Banking Online\xa00800 169 1296+44 1293 762 380 from abroad.\xa0Lines are open 24 hours.You'll need to tell us the reason for your call so

In [16]:
# couldnt get langchain to work so custom RAG anyone?
import time

class RAG:
    def __init__(self, vector_store, palm_wrapper, initial_system_prompt=True):
        self.vector_store = vector_store
        self.palm_wrapper = palm_wrapper
        # the comment from mathew regarding memory
        self.conversation_history = []
        
        if initial_system_prompt:            
            system_prompt = r"You are a professional assistant with extensive experience helping numerous small and medium businesses. You work for a large retail bank called Lloyds. Please assist the user answering questions with detailed responses, providing reasoning whenever prescriptive advice is given."
            self.conversation_history.append((system_prompt, ""))
        
    
    def generate(self, query, is_user_query=True, k=3):
        vector_start_time = time.time()
        
        contexts=[]
        sources=[]
        scores=[]
        
        if is_user_query:
            # if user query, perform vector db search
            # retrieve contexts based on the query
            search_results = self.vector_store.search(query, k)
            contexts = [result[1] for result in search_results]
            sources = [result[0] for result in search_results]
            scores = [result[2] for result in search_results]
        
        vector_end_time = time.time()
        # combine both query and response from conversation history
        history_context = '\n'.join(['Q: ' + query + '\nA: ' + response for query, response in self.conversation_history[-k:]])
        
        # combine history context and current contexts
        combined_context = '\n'.join([history_context] + contexts)
        
        lm_start_time = time.time()
        response = self.palm_wrapper.generate_response(combined_context[:20000], query)
        lm_end_time = time.time()
        # update conversation history with current interaction
        self.conversation_history.append((query, response))
        
        vector_search_time = vector_end_time - vector_start_time
        lm_inference_time = lm_end_time - lm_start_time
        
        return response, sources, scores, vector_search_time, lm_inference_time

In [17]:
rag = RAG(vai, pw)
query = "how do i change my address?"
response, sources, scores, vector_search_time, lm_inference_time = rag.generate(query)
print(response)
print('sources used:', sources)
print('scores:', scores)

 To change your address, you can either update it online or by form. 

*If you’re a full access user, you can update your business, registered, mailing and personal address.*

To do this online, you will need your card reader ready to approve your changes.

1. Log into Online for Business
2. Select Admin
3. In the Business Details section, select Your addresses
4. This screen displays your address details.  
5. Find the address you want to update and select Change
6. Follow the onscreen guidance
7. You’ll need to verify yourself using your card and reader once you’ve made your changes.  
8. You’ll then get a reference number.

*Alternatively, you can also change your address by form.*

To get you to the right form, Lloyds asks a few questions: 

1. Do you have a named Relationship Manager? 
2. Is your turnover over £25 million? 

Based on your answers, you will be directed to the appropriate form and instructions on how to submit it.
sources used: ['24', '26', '45']
scores: [0.71750628

In [18]:
rag = RAG(vai, pw)
query = "i've lost my card"
response, sources, scores, vector_search_time, lm_inference_time = rag.generate(query)
print(response)
print('sources used:', sources)
print('scores:', scores)
print('vector search time:', vector_search_time)
print('llm inference time:', lm_inference_time)

query = "why?"
response, sources, scores, vector_search_time, lm_inference_time = rag.generate(query)
print(response)
print('sources used:', sources)
print('scores:', scores)
print('vector search time:', vector_search_time)
print('llm inference time:', lm_inference_time)

query = "i was not asking about an audit request"
response, sources, score, vector_search_time, lm_inference_times = rag.generate(query)
print(response)
print('sources used:', sources)
print('scores:', scores)
print('vector search time:', vector_search_time)
print('llm inference time:', lm_inference_time)

query = "what is the weather in london"
response, sources, scores, vector_search_time, lm_inference_time = rag.generate(query)
print(response)
print('sources used:', sources)
print('scores:', scores)
print('vector search time:', vector_search_time)
print('llm inference time:', lm_inference_time)

query = "how can i set up an online banking account?"
response, sources, scores, vector_search_time, lm_inference_time = rag.generate(query)
print(response)
print('sources used:', sources)
print('scores:', scores)
print('vector search time:', vector_search_time)
print('llm inference time:', lm_inference_time)

 If your Corporate Card has been lost, stolen or misused please call us right away. This applies to your PIN or security details too.
Payment cards include:
Corporate MultiPay card
Corporate Charge Card
Corporate Purchasing Card
Business Travel Solution
ePay Virtual
ePayables
By Phone
Call us anytime on 0800 096 4496
If you're outside the UK call +44 1908 544 059.
sources used: ['56', '53', '62']
scores: [0.7142672538757324, 0.7047110795974731, 0.7031532526016235]
vector search time: 0.21158623695373535
llm inference time: 2.4716455936431885
 
sources used: ['50', '51', '16']
scores: [0.6711143851280212, 0.6682686805725098, 0.6521599292755127]
vector search time: 0.42149877548217773
llm inference time: 2.6167328357696533
 Apologies for the confusion. 

Do you want to know how to report a lost card, or why you need to report a lost card?

sources used: ['50', '16', '67']
scores: [0.6711143851280212, 0.6682686805725098, 0.6521599292755127]
vector search time: 0.21642374992370605
llm infe

In [19]:
import ipywidgets as widgets
from IPython.display import display, clear_output

class ChatUI:
    def __init__(self, rag_instance):
        self.rag_instance = rag_instance
        self.conversation = []
        self._setup_ui()
    
    def _setup_ui(self):
        self.input_box = widgets.Text(
            placeholder='Message LBG help and support...',
            description='Lloyds',
            layout={'width': '80%'}
        )
        
        self.send_button = widgets.Button(
            description='Send',
            button_style='info',
            layout={'width': '12%'}
        )
        self.output_area = widgets.Output(layout={'border': '1px solid black', 'width': '100%'})
        self.send_button.on_click(self._on_send_clicked)
        
        input_send_box = widgets.HBox([self.input_box, self.send_button])
        
        display(self.output_area, input_send_box)
        
    def _on_send_clicked(self, b):
        query = self.input_box.value
        self.conversation.append(f"You: {query}")
        response = self.rag_instance.generate(query)
        self.conversation.append(f'LBG AI: {response}')
        
        with self.output_area:
            clear_output(wait=True)
            print('\n'.join(self.conversation))
        
        # clear input box
        self.input_box.value = ''
        
rag = RAG(vai, pw)
chat_ui = ChatUI(rag)            

Output(layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right='1px solid b…